In [ ]:
# Initializing NumPy
import numpy as np
import os
import psutil
from time import time
import gc
import sqlite3
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filename = '/content/drive/My Drive/DataEngg/WineQT.sqlite'
csv_file = "/content/drive/My Drive/DataEngg/WineQT.csv"
conn = sqlite3.connect(filename)
cursor = conn.cursor()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/DataEngg"

WineQT.csv  WineQT.sqlite


In [ ]:
# Defining data types for each column
dtype = np.dtype([
    ('fixed acidity', float),
    ('volatile acidity', float),
    ('citric acid', float),
    ('residual sugar', float),
    ('chlorides', float),
    ('free sulfur dioxide', float),
    ('total sulfur dioxide', float),
    ('density', float),
    ('pH', float),
    ('sulphates', float),
    ('alcohol', float),
    ('Id', int),
    ('quality', int)
])


# Loading dataset from CSV file
# data = np.genfromtxt("/content/drive/My Drive/DataEngg/WineQT.csv", delimiter=',', dtype=dtype, names=True, encoding=None)

In [ ]:
#    Measures and prints the time and memory used for a given operation on a dataset.
def measure_performance(operation, data=None):
    gc.collect()
    gc.disable()  # to prevent it from interfering during the operation

    start_time = time()
    process = psutil.Process(os.getpid())
    start_memory = process.memory_info().rss

    if data is not None:
        result = operation(data)
    else:
        result = operation()

    elapsed_time = time() - start_time
    end_memory = process.memory_info().rss

    gc.enable()  # Re-enable after the operation
    gc.collect()  # clean up after enabling

    memory_used = (end_memory - start_memory) / (1024 * 1024)
    # / (1024 * 1024)

    print("Time to complete operation: {} seconds".format(round(elapsed_time, 3)))
    print("Memory used: {} MB".format(round(memory_used, 3)))

    return result

### **LOAD CSV**

In [ ]:
def load_data():
    return np.genfromtxt(csv_file, delimiter=',', dtype=dtype, names=True, encoding=None)

data = measure_performance(load_data)


Time to complete operation: 0.307 seconds
Memory used: 1.805 MB


In [ ]:
def load_data_sqlite():
    df = pd.read_csv(csv_file)

    df.to_sql('wine_data', conn, if_exists='replace', index=False, method='multi')

    conn.commit()
    # conn.close()

# Measure the performance of loading data into SQLite
measure_performance(load_data_sqlite)


Time to complete operation: 0.071 seconds
Memory used: 4.246 MB


In [ ]:
def query_numpy(data):
    return len(data)

def query_sqlite():
    cursor.execute("SELECT COUNT(*) FROM wine_data")
    result = cursor.fetchone()[0]

    conn.close()
    return result

print("NumPy row count:", measure_performance(query_numpy, data))
conn = sqlite3.connect(filename)
cursor = conn.cursor()

print("SQLite row count:", measure_performance(query_sqlite))


Time to complete operation: 0.0 seconds
Memory used: 0.0 MB
NumPy row count: 1143
Time to complete operation: 0.004 seconds
Memory used: 0.0 MB
SQLite row count: 1143


In [ ]:
# Checking to see if the data has been correctly loaded
# print(data)
print(data.dtype.names)

print(f"Number of elements: {data.size}")

('fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality', 'Id')
Number of elements: 1143


In [ ]:
def check_table_schema():
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()

    cursor.execute("PRAGMA table_info(wine_data)")
    columns = cursor.fetchall()
    for column in columns:
        print(column)

    conn.close()

check_table_schema()


(0, 'fixed acidity', 'REAL', 0, None, 0)
(1, 'volatile acidity', 'REAL', 0, None, 0)
(2, 'citric acid', 'REAL', 0, None, 0)
(3, 'residual sugar', 'REAL', 0, None, 0)
(4, 'chlorides', 'REAL', 0, None, 0)
(5, 'free sulfur dioxide', 'REAL', 0, None, 0)
(6, 'total sulfur dioxide', 'REAL', 0, None, 0)
(7, 'density', 'REAL', 0, None, 0)
(8, 'pH', 'REAL', 0, None, 0)
(9, 'sulphates', 'REAL', 0, None, 0)
(10, 'alcohol', 'REAL', 0, None, 0)
(11, 'quality', 'INTEGER', 0, None, 0)
(12, 'Id', 'INTEGER', 0, None, 0)


## Filtering Data

In [ ]:
def filter_data_numpy(data):
    return (data[(data['volatile_acidity'] < 0.7) &
                        (data['alcohol'] > 10) &
                        (data['quality'] >= 5)])


def filter_data_sqlite():
  cursor.execute("""
    SELECT * FROM wine_data
    WHERE "volatile acidity" < 0.7 AND alcohol > 10 AND quality >= 5
    """)
  results = cursor.fetchall()
  return results

print("NumPy filtering: ")
filter_data = measure_performance(filter_data_numpy, data)
conn = sqlite3.connect(filename)
cursor = conn.cursor()

print("SQLite filtering: ", measure_performance(filter_data_sqlite))



NumPy filtering: 
Time to complete operation: 0.002 seconds
Memory used: 0.0 MB
Time to complete operation: 0.005 seconds
Memory used: 0.0 MB
SQLite filtering:  [(8.5, 0.28, 0.56, 1.8, 0.092, 35.0, 103.0, 0.9969, 3.3, 0.75, 10.5, 7, 16), (6.7, 0.675, 0.07, 2.4, 0.089, 17.0, 82.0, 0.9958, 3.35, 0.54, 10.1, 5, 30), (7.8, 0.6, 0.14, 2.4, 0.086, 3.0, 15.0, 0.9975, 3.42, 0.6, 10.8, 6, 36), (7.3, 0.45, 0.36, 5.9, 0.074, 12.0, 87.0, 0.9978, 3.33, 0.83, 10.5, 5, 40), (7.5, 0.49, 0.2, 2.6, 0.332, 8.0, 14.0, 0.9968, 3.21, 0.9, 10.5, 6, 42), (8.1, 0.66, 0.22, 2.2, 0.069, 9.0, 23.0, 0.9968, 3.3, 1.2, 10.3, 5, 43), (7.6, 0.51, 0.15, 2.8, 0.11, 33.0, 73.0, 0.9955, 3.17, 0.63, 10.2, 6, 54), (8.8, 0.41, 0.64, 2.2, 0.093, 9.0, 42.0, 0.9986, 3.54, 0.66, 10.5, 5, 76), (6.3, 0.3, 0.48, 1.8, 0.069, 18.0, 61.0, 0.9959, 3.44, 0.78, 10.3, 6, 84), (6.9, 0.55, 0.15, 2.2, 0.076, 19.0, 40.0, 0.9961, 3.41, 0.59, 10.1, 5, 85), (8.3, 0.61, 0.3, 2.1, 0.084, 11.0, 50.0, 0.9972, 3.4, 0.61, 10.2, 6, 100), (9.4, 0.4, 0.3

In [ ]:
# large_data = np.tile(data, (1000, 1))
# print(large_data.size)


def create_large_dataset_numpy(data):
    return np.tile(data, (1000, 1))

print("Time to create large dataset with NumPy: ")

large_data = measure_performance(create_large_dataset_numpy, data)

# measure_performance(create_large_dataset_numpy, data)
# print("Size of large dataset:", large_data_numpy.size)

# filtered_large_data = measure_performance(filter_data, large_data)


Time to create large dataset with NumPy: 
Time to complete operation: 0.048 seconds
Memory used: 113.43 MB


In [ ]:
conn = sqlite3.connect(filename)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS wine_data_large AS SELECT * FROM wine_data WHERE 1=0;")

print("Time to create large dataset with SQLite: ")

def repeat_data_insertion():
    for _ in range(999):

        cursor.execute("""
        INSERT INTO wine_data_large SELECT * FROM wine_data
        """)
    conn.commit()

# Scale the data
large_data_sql = measure_performance(repeat_data_insertion)


Time to create large dataset with SQLite: 
Time to complete operation: 3.986 seconds
Memory used: 0.0 MB


### Sorting

In [ ]:
# def sortByQuality(data):
#   return np.sort(data, order=['quality', 'alcohol'])

# large_sorted_by_quality_alcohol = measure_performance(sortByQuality, data)
# large_sorted_by_quality_alcohol = measure_performance(sortByQuality, large_data)


In [ ]:
def aggregate_alcohol_by_quality_numpy(data):
    qualities = np.unique(data['quality'])
    return {quality: data['alcohol'][data['quality'] == quality].mean() for quality in qualities}


def aggregate_alcohol_by_quality_sqlite():
    cursor.execute("""
        SELECT quality, AVG(alcohol) AS avg_alcohol
        FROM wine_data
        GROUP BY quality
    """)
    results = cursor.fetchall()
    return {result[0]: result[1] for result in results}

conn = sqlite3.connect(filename)
cursor = conn.cursor()

print("SQLite performance of alcohol aggregation: ")
alcohol_aggregation_sqlite = measure_performance(aggregate_alcohol_by_quality_sqlite)

conn.close()
print("numPy performance of alcohol aggregation: ")
alcohol_aggregation = measure_performance(aggregate_alcohol_by_quality_numpy, data)
# alcohol_aggregation = measure_performance(aggregate_alcohol_by_quality_numpy, large_data)


SQLite performance of alcohol aggregation: 
Time to complete operation: 0.002 seconds
Memory used: 0.0 MB
numPy performance of alcohol aggregation: 
Time to complete operation: 0.005 seconds
Memory used: 0.0 MB


In [ ]:
def multi_aggregation(data):
    result = {
        'mean_alcohol': np.mean(data['alcohol']),
        'max_alcohol': np.max(data['alcohol']),
        'min_alcohol': np.min(data['alcohol']),
        'average_density': np.mean(data['density']),
        'highest_sulfur': np.max(data['total_sulfur_dioxide']),
        'lowest_pH': np.min(data['pH'])
    }
    return result


def multi_aggregation_sqlite():

    cursor.execute("""
        SELECT
            AVG(alcohol) AS mean_alcohol,
            MAX(alcohol) AS max_alcohol,
            MIN(alcohol) AS min_alcohol,
            AVG(density) AS average_density,
            MAX("total sulfur dioxide") AS highest_sulfur,
            MIN(pH) AS lowest_pH
        FROM wine_data
    """)
    result = cursor.fetchone()

    return {
        'mean_alcohol': result[0],
        'max_alcohol': result[1],
        'min_alcohol': result[2],
        'average_density': result[3],
        'highest_sulfur': result[4],
        'lowest_pH': result[5]
    }


print(measure_performance(multi_aggregation, data))
# print(measure_performance(multi_aggregation, large_data))
conn = sqlite3.connect(filename)
cursor = conn.cursor()
print(measure_performance(multi_aggregation_sqlite))
conn.close()




Time to complete operation: 0.005 seconds
Memory used: 0.0 MB
{'mean_alcohol': 10.442111402741325, 'max_alcohol': 14.9, 'min_alcohol': 8.4, 'average_density': 0.9967304111986001, 'highest_sulfur': 289.0, 'lowest_pH': 2.74}
Time to complete operation: 0.003 seconds
Memory used: 0.0 MB
{'mean_alcohol': 10.44211140274131, 'max_alcohol': 14.9, 'min_alcohol': 8.4, 'average_density': 0.9967304111986008, 'highest_sulfur': 289.0, 'lowest_pH': 2.74}
